In [6]:
import requests, time, os, json

In [7]:
# 1) Configure
object_ids = ["103SPM", "109Q8G", "107HB7"]
iiif_base   = "https://data.getty.edu/museum/api/iiif"
out_dir     = "greek_statues"
os.makedirs(out_dir, exist_ok=True)

for oid in object_ids:
    manifest_url = f"{iiif_base}/{oid}/manifest.json"
    r = requests.get(manifest_url)
    r.raise_for_status()           # will stop if anything goes wrong

    manifest = r.json()

    # 2) Extract core metadata
    title = manifest.get("label", "Untitled")
    meta  = {m["label"]: m["value"] for m in manifest.get("metadata", [])}
    date        = meta.get("Date", meta.get("Display Date", "Unknown"))
    description = meta.get("Description", "No description available")

    print(f"{oid} → {title} ({date})")

    # 3) Pull the IIIF Image service URL
    svc = (
        manifest["sequences"][0]
                ["canvases"][0]
                ["images"][0]
                ["resource"]
                ["service"]
                ["@id"]
    )

    # 4) Download an 800 px-wide JPEG
    img_url = f"{svc}/full/800,/0/default.jpg"
    img_data = requests.get(img_url).content
    img_path = os.path.join(out_dir, f"{oid}.jpg")
    with open(img_path, "wb") as img_f:
        img_f.write(img_data)

    # 5) Save metadata to a JSON file alongside
    meta_path = os.path.join(out_dir, f"{oid}.json")
    with open(meta_path, "w", encoding="utf-8") as json_f:
        json.dump({
            "id": oid,
            "title": title,
            "date": date,
            "description": description,
            "iiif_service": svc
        }, json_f, indent=2, ensure_ascii=False)

    print(f" • Saved image → {img_path}")
    print(f" • Saved metadata → {meta_path}\n")


HTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://data.getty.edu/museum/api/iiif/103SPM/manifest.json